In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import timm
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import timm
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import glob
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
!pip install timm

In [ ]:
class EggDataset(Dataset):
    def __init__(self, tt,transform=None):
        self.image_paths = "/content/drive/MyDrive/eggs/"+tt
        self.transform = transform
        file_list = glob.glob(self.image_paths + "*")
        print(file_list)
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        print(self.data)
        self.class_map = {"0" : 0, "1": 1, "2": 2, "3": 3}
        self.img_dim = (416, 416)
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        class_id = self.class_map[class_name]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        return image, class_id

In [ ]:
test_transforms = A.Compose(
    [
        A.SmallestMaxSize(max_size=350),
        A.CenterCrop(height=256, width=256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

In [ ]:
train_dataset = EggDataset('train/',test_transforms )
test_dataset = EggDataset('test/',test_transforms )
train_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=0)

['/content/drive/MyDrive/eggs/train/0', '/content/drive/MyDrive/eggs/train/1', '/content/drive/MyDrive/eggs/train/2', '/content/drive/MyDrive/eggs/train/3']
[['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172433.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172438.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172441.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172446.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172448.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172454.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172501.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172510.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172512.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172515.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172517.jpg', '0'], ['/content/drive/MyDrive/eggs/train/0/IMG_20180213_172519.jpg', '0'], ['

In [ ]:
lr = 0.001
num_epochs = 10
device = 'cuda'

In [ ]:
model = timm.create_model('resnet18d', pretrained=True)#SimpleConvNet()
model.fc = nn.Linear(512, 4)
model.to(device)
None

In [ ]:
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

In [ ]:
sum(i.numel() for i in model.parameters()) / 1000000

11.197796

In [ ]:
loss_fun = CrossEntropyLoss()

In [ ]:
len(test_dataset)

65

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, total_steps=len(train_loader) * num_epochs, pct_start=0.2)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader):
        model.zero_grad()
        x, labels = batch
        x, labels = x.to(device), labels.to(device)
        out = model(x)
        loss = loss_fun(out, labels)
        #print(loss)
        loss.backward()
        optimizer.step()
        scheduler.step()
        print({'train_loss': loss.item()})
    model.eval()
    all_labels = []
    all_preds = []
    for batch in tqdm(test_loader):
        with torch.no_grad():
            x, labels = batch
            all_labels += list(labels.numpy())
            x, labels = x.to(device), labels.to(device)
            out = model(x)
            all_preds += list(out.argmax(dim=1).to('cpu').numpy())
            loss = loss_fun(out, labels)
    print({'val_loss': loss.item(), 'accuracy': accuracy_score(all_labels, all_preds)})

  0%|          | 0/50 [00:00<?, ?it/s]

{'train_loss': 1.3138213157653809}
{'train_loss': 1.2502168416976929}
{'train_loss': 1.4558155536651611}
{'train_loss': 1.2284619808197021}
{'train_loss': 1.2635059356689453}
{'train_loss': 1.2910773754119873}
{'train_loss': 1.5897430181503296}
{'train_loss': 1.3626140356063843}
{'train_loss': 1.3102173805236816}
{'train_loss': 1.338423490524292}
{'train_loss': 1.384118914604187}
{'train_loss': 1.293999195098877}
{'train_loss': 1.28472900390625}
{'train_loss': 1.2390698194503784}
{'train_loss': 1.0517621040344238}
{'train_loss': 1.037148118019104}
{'train_loss': 1.1910507678985596}
{'train_loss': 1.1539591550827026}
{'train_loss': 1.4475853443145752}
{'train_loss': 1.2245771884918213}
{'train_loss': 1.042291522026062}
{'train_loss': 1.2292351722717285}
{'train_loss': 1.1237331628799438}
{'train_loss': 0.9938122034072876}
{'train_loss': 0.9025506377220154}
{'train_loss': 0.8614773750305176}
{'train_loss': 0.882413387298584}
{'train_loss': 0.8692653179168701}
{'train_loss': 1.10286939144

  0%|          | 0/17 [00:00<?, ?it/s]

{'val_loss': 0.7494208812713623, 'accuracy': 0.6923076923076923}


  0%|          | 0/50 [00:00<?, ?it/s]

...

{'val_loss': 0.04733743518590927, 'accuracy': 0.8615384615384616}


  0%|          | 0/50 [00:00<?, ?it/s]

{'train_loss': 0.05042493715882301}
{'train_loss': 0.019194912165403366}
{'train_loss': 0.03840918093919754}
{'train_loss': 0.05373678728938103}
{'train_loss': 0.010635814629495144}
{'train_loss': 0.06495988368988037}
{'train_loss': 0.013673627749085426}
{'train_loss': 0.004708257969468832}
{'train_loss': 0.00580102764070034}
{'train_loss': 0.040724337100982666}
{'train_loss': 0.10430382192134857}
{'train_loss': 0.011303098872303963}
{'train_loss': 0.00800367258489132}
{'train_loss': 0.0027376855723559856}
{'train_loss': 0.007249115034937859}
{'train_loss': 0.010476198978722095}
{'train_loss': 0.3793891966342926}
{'train_loss': 0.004089018329977989}
{'train_loss': 0.005805969703942537}
{'train_loss': 0.010724823921918869}
{'train_loss': 0.16414546966552734}
{'train_loss': 0.002503063064068556}
{'train_loss': 0.011881262063980103}
{'train_loss': 0.005900419782847166}
{'train_loss': 0.009421570226550102}
{'train_loss': 0.008763040415942669}
{'train_loss': 0.08662810176610947}
{'train_los

  0%|          | 0/17 [00:00<?, ?it/s]

{'val_loss': 4.355116844177246, 'accuracy': 0.8461538461538461}


In [ ]:
#проверяем работу модели
test_image = cv2.imread("/content/1.jpg")
model.zero_grad()
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
test_image = test_transforms(image=test_image)["image"]
test_image=test_image.unsqueeze(0)
device = 'cuda'
test_image=test_image.to(device)
prediction = model(test_image)
str(torch.argmax(prediction, dim=-1)[0])


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/eggs/eggs1.pth")

In [ ]:
#проверяем загрузку модели
device = 'cuda'
model = timm.create_model('resnet18d', pretrained=True)  # SimpleConvNet()
model.fc = nn.Linear(512, 4)
model.to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/eggs/eggs1.pth'))

<All keys matched successfully>